<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

# A1: Regression Model Development

**Author:** Max Lembke 

**Course:** Machine Learning - DAT-5303

**Date:** 02/09/2021

<br><hr style="height:.9px;border:none;color:#333;background-color:#333;" />

##  1) Set-up & Data Preparation 

### a) Import of Libraries & File 

In [1]:
# importing libraries
import pandas as pd # data science essentials
import numpy as np #math esstentials 
import matplotlib.pyplot as plt # data visualization
import seaborn as sns # enhanced data visualization
import statsmodels.formula.api as smf # regression modeling
from sklearn.model_selection import train_test_split # train/test split
from sklearn.linear_model import LinearRegression #Linear Regression
import sklearn.linear_model # linear models
from sklearn.linear_model import Lasso  # Lasso Regression
from sklearn.linear_model import ARDRegression # ARD Regression
from sklearn.neighbors import KNeighborsRegressor # KNeighborsRegressor
from sklearn.preprocessing import StandardScaler # StandardScaler

# setting pandas print options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# importing file 
file = "./Apprentice_Chef_Dataset.xlsx"

# reading file 
df_all = pd.read_excel(file)

# lowercase column names to make typing in the future easier
df_all.columns = map(str.lower, df_all.columns)

# renaming mislabled 
df_all = df_all.rename(columns = {'largest_order_size':'avg_num_meals_customer'})


# checking for nulls and imputing 0 
#df_all.isnull().sum(axis = 0) 
df_all = df_all.fillna(0) #impute 0 for no data
#df_all.isnull().sum(axis = 0) #check if removed

# show head to verify renaming, loading of data and other transformations worked 
df_all.head(n=5)


,revenue,cross_sell_success,name,email,first_name,family_name,total_meals_ordered,unique_meals_purch,contacts_w_customer_service,product_categories_viewed,avg_time_per_site_visit,mobile_number,cancellations_before_noon,cancellations_after_noon,tastes_and_preferences,pc_logins,mobile_logins,weekly_plan,early_deliveries,late_deliveries,package_locker,refrigerated_locker,avg_prep_vid_time,avg_num_meals_customer,master_classes_attended,median_meal_rating,avg_clicks_per_visit,total_photos_viewed
0,393.0,1,Saathos,saathos@unitedhealth.com,Saathos,Saathos,14,6,12,10,48.00,1,3,1,1,5,2,0,0,2,0,0,33.4,1,0,1,17,0
1,1365.0,1,Alysanne Osgrey,alysanne.osgrey@ge.org,Alysanne,Osgrey,87,3,8,8,40.35,1,0,0,1,5,1,12,0,2,0,0,84.8,1,0,3,13,170
2,800.0,1,Edwyd Fossoway,edwyd.fossoway@jnj.com,Edwyd,Fossoway,15,7,11,5,19.77,1,3,0,1,6,1,1,0,1,0,0,63.0,1,0,2,16,0
3,600.0,1,Eleyna Westerling,eleyna.westerling@ge.org,Eleyna,Westerling,13,6,11,5,90.00,1,2,0,1,6,1,14,0,3,0,0,43.8,1,0,2,14,0
4,1490.0,1,Elyn Norridge,elyn.norridge@jnj.com,Elyn,Norridge,47,8,6,10,40.38,1,0,0,0,5,1,5,0,8,0,0,84.8,1,1,3,12,205


### b) Initial Exploration & Sample Log Transformation 

In [ ]:
# quick and dirty exploration of all the variables 
# commented out as not needed 

#df_all.describe()

# creating a new df excluding string cloumns 
#df =  df_all.loc[:, ~df_all.columns.isin(["name", "email", "first_name", "family_name"])]

# loop to plot displots of all non-string columns
#for i, column in enumerate(df.columns):
   #sns.distplot(df[column]) 
   #plt.show()
    
# displot of revenue as example 

sns.distplot(a = df_all['revenue'])

In [ ]:
# log transformation of revenue 

df_all['log_revenue'] = np.log10(df_all['revenue'])

sns.distplot(a = df_all['log_revenue'])

### c) Feature Engineering 

In [ ]:
## Calculated Features ##

# total logins 
df_all['total_logins'] = df_all['pc_logins'] + df_all['mobile_logins']

# total cancellations 
df_all['total_cancel'] = df_all['cancellations_before_noon'] + df_all['cancellations_after_noon']

# every cancelled y/n

df_all['every_cancelled'] = 0  #pre-allocating 
df_all.loc[(df_all['total_cancel']>0.5), 'every_cancelled' ] = 1

# total orders without cancellations 
df_all['total_meals_ordered_without_cancel'] = df_all['total_meals_ordered'] - df_all['total_cancel']

# order variety 
df_all['order_variety'] = df_all['total_meals_ordered'] / df_all['unique_meals_purch'] 

# amount of late to total 
df_all['late_ratio_total'] = df_all['late_deliveries'] / df_all['total_meals_ordered']

# angry customer due to late meal
df_all['contact_service_late'] = df_all['late_deliveries'] / df_all['contacts_w_customer_service'] 

# amount of customer service contact 
df_all['contact_service_orders'] = df_all['contacts_w_customer_service'] / df_all['total_meals_ordered']

# logins to orders 
df_all['meals_per_login'] = df_all['total_meals_ordered'] / df_all['total_logins'] 

# clicks to order 
df_all['avg_num_meals_per_click'] = df_all['avg_num_meals_customer'] /df_all['avg_clicks_per_visit'] 

# clicks to order 
df_all['orders_per_login'] = df_all['total_meals_ordered'] / df_all['total_logins']

# views to unique orders 
df_all['meals_per_pic_view'] = df_all['unique_meals_purch'] / df_all['total_photos_viewed'] 

# total lockers (and w/ y/n)
df_all['total_lockers'] = df_all['package_locker'] + df_all['package_locker'] 

df_all['lockers_present'] = 0  #pre-allocating 
df_all.loc[(df_all['total_lockers']>0.5), 'lockers_present' ] = 1

# order time change (and w/ y/n)
df_all['order_time_change'] = df_all['early_deliveries'] + df_all['late_deliveries']

df_all['order_time_change_ever'] = 0  #pre-allocating 
df_all.loc[(df_all['order_time_change']>0.5), 'order_time_change_ever' ] = 1

# weekly plan y/n
df_all['weekly_plan_present'] = 0  #pre-allocating 
df_all.loc[(df_all['weekly_plan']>0.5), 'weekly_plan_present' ] = 1

# less than 30 order flag 
df_all['less_than_30_flag'] = 0  #pre-allocating 
df_all.loc[(df_all['weekly_plan']<30), 'less_than_30_flag' ] = 1

In [ ]:
## Mail Feature ## 

# placeholder list
placeholder_lst = []

# looping over each email address
for index, col in df_all.iterrows():
    
    # splitting email domain at '@'
    split_email = df_all.loc[index, 'email'].split(sep = '@')
    
    # appending placeholder_lst with the results
    placeholder_lst.append(split_email)
    

# converting placeholder_lst into a DataFrame 
email_df = pd.DataFrame(placeholder_lst)

# renaming column to concatenate
email_df.columns = ['0' , 'domain']

# concatenating domain with df_all
df_all = pd.concat([df_all, email_df['domain']],
                     axis = 1)

# creating column to be allocated with email type 
df_all['mail_type'] = '0'


# defintion of email types 

prof = ['walmart.com','mmm.com', 'pg.com',
        'amex.com', 'apple.com','boeing.com', 
        'caterpillar.com', 'chevron.com',
        'cisco.com', 'cocacola.com', 'disney.com', 
        'dupont.com', 'exxon.com', 'ge.org', 
        'goldmansacs.com', 'homedepot.com', 'ibm.com', 
        'intel.com', 'jnj.com', 'jpmorgan.com', 
        'mcdonalds.com', 'merck.com', 'microsoft.com', 
        'nike.com', 'pfizer.com', 
        'travelers.com', 'unitedtech.com', 'unitedhealth.com',
        'verizon.com', 'visa.com']

pers = ['gmail.com','protonmail.com','yahoo.com','aol.com']

spam = ["hotmail.com","me.com", "msn.com","live.com", "passport.com"]

# loop for classification 

for index, row in df_all.iterrows():
    if df_all.loc[index,'domain'] in prof: 
        df_all.loc[index,'mail_type'] = 'professional'
        
    elif df_all.loc[index,'domain'] in pers: 
        df_all.loc[index,'mail_type'] = 'personal'
        
    elif df_all.loc[index,'domain'] in spam: 
        df_all.loc[index,'mail_type'] = 'spam'
        
    elif df_all.loc[index, 'domain'] not in prof or df_all.loc[
            row, 'domain'] not in pers or df_all.loc[row, 'domain'] not in spam:
        df_all.loc[index, 'mail_type'] = 'unknown'
        
    else:
        print('Mistake')
    

# creating dummies based on email type 
mail_dummies = pd.get_dummies(df_all.mail_type, prefix = "dum_mail_type",drop_first = True)

# adding dummies to main dataframe 
df_all = pd.concat([df_all,mail_dummies], axis = 1)

In [ ]:
## Dummies ## 

# creating dummies based on rating
rating_dummies = pd.get_dummies(df_all.median_meal_rating,prefix = "dum_median_rating",drop_first = True)

# adding dummies to main dataframe 
df_all = pd.concat([df_all,rating_dummies], axis = 1)

# creating dummies based on classes attended
attended_dummies = pd.get_dummies(df_all.master_classes_attended,prefix = "dum_master_classes_attended", drop_first = True)

# adding dummies to main dataframe 
df_all = pd.concat([df_all,attended_dummies], axis = 1)

In [ ]:
## Re-check distributions ## 

# creating a new df excluding string cloumns 
df =  df_all.loc[:, ~df_all.columns.isin(["name", "email", "first_name", "family_name", "domain", "mail_type"])]

# loop to plot displots of all non-string columns
#for i, column in enumerate(df.columns):
   #sns.distplot(df[column]) 
   #plt.show()

In [ ]:
## Log transformation ##

# Changing 0 to 0.00000001 to avoid devide by zeros 

df_all['cancellations_before_noon'] = df_all['cancellations_before_noon'].replace(0,0.00000001)
df_all['cancellations_after_noon'] = df_all['cancellations_after_noon'].replace(0,0.00000001)
df_all['weekly_plan'] = df_all['weekly_plan'].replace(0,0.00000001)
df_all['early_deliveries'] = df_all['early_deliveries'].replace(0,0.00000001)
df_all['late_deliveries'] = df_all['late_deliveries'].replace(0,0.00000001)
df_all['total_photos_viewed'] = df_all['total_photos_viewed'].replace(0,0.00000001)
df_all['total_cancel'] = df_all['total_cancel'].replace(0,0.00000001)
df_all['late_ratio_total'] = df_all['late_ratio_total'].replace(0,0.00000001)
df_all['contact_service_late'] = df_all['contact_service_late'].replace(0,0.00000001)

# log transformations
df_all['log_total_meals_ordered'] = np.log10(df_all['total_meals_ordered'])
df_all['log_unique_meals_purch'] = np.log10(df_all['unique_meals_purch'])
df_all['log_contacts_w_customer_service'] = np.log10(df_all['contacts_w_customer_service'])
df_all['log_product_categories_viewed'] = np.log10(df_all['product_categories_viewed'])
df_all['log_avg_time_per_site_visit'] = np.log10(df_all['avg_time_per_site_visit'])
df_all['log_cancellations_before_noon'] = np.log10(df_all['cancellations_before_noon'])
df_all['log_cancellations_after_noon'] = np.log10(df_all['cancellations_after_noon'])
df_all['log_weekly_plan'] = np.log10(df_all['weekly_plan'])
df_all['log_early_deliveries'] = np.log10(df_all['early_deliveries'])
df_all['log_late_deliveries'] = np.log10(df_all['late_deliveries'])
df_all['log_avg_prep_vid_time'] = np.log10(df_all['avg_prep_vid_time'])
df_all['log_avg_num_meals_customer'] = np.log10(df_all['avg_num_meals_customer'])
df_all['log_median_meal_rating'] = np.log10(df_all['median_meal_rating'])
df_all['log_total_photos_viewed'] = np.log10(df_all['total_photos_viewed'])
df_all['log_total_logins'] = np.log10(df_all['total_logins'])
df_all['log_total_cancel'] = np.log10(df_all['total_cancel'])
df_all['log_total_meals_ordered_without_cancel'] = np.log10(df_all['total_meals_ordered_without_cancel'])
df_all['log_order_variety'] = np.log10(df_all['order_variety'])
df_all['log_late_ratio_total'] = np.log10(df_all['late_ratio_total'])
df_all['log_contact_service_late'] = np.log10(df_all['contact_service_late'])
df_all['log_contact_service_orders'] = np.log10(df_all['contact_service_orders'])
df_all['log_avg_num_meals_per_click'] = np.log10(df_all['avg_num_meals_per_click'])


In [ ]:
df_all.head(n=5)

## 2) Model

### a) Preparation and Base Model

#### i) Removing columns and simple correlation 

In [ ]:
# removing columns not needed 

delete = ["revenue","name", "email", "first_name", "family_name", "domain", "mail_type"]
df = df_all.drop(delete, axis = 1)

df_corr = df.corr(method = 'pearson')

df_corr.loc[ : , "log_revenue"].round(decimals = 2).sort_values(ascending = False)

#### ii) X Variable Definition 

In [ ]:
# Defining all the x variables to avoid dublicates x and log 

x_variables = ['dum_master_classes_attended_1',
'dum_master_classes_attended_2',
'dum_master_classes_attended_3',
'dum_median_rating_2',
'dum_median_rating_3',
'dum_median_rating_4',
'dum_median_rating_5',
'dum_mail_type_unknown',
'dum_mail_type_spam',
'dum_mail_type_professional',
'log_total_meals_ordered',
'log_total_meals_ordered_without_cancel',
'log_avg_prep_vid_time',
'log_order_variety',
'log_median_meal_rating',
'orders_per_login',
'meals_per_login',
'avg_num_meals_per_click',
'order_variety',
'log_avg_num_meals_customer',
'total_photos_viewed',
'log_avg_time_per_site_visit',
'log_contacts_w_customer_service',
'mobile_number',
'product_categories_viewed',
'log_cancellations_before_noon',
'tastes_and_preferences',
'pc_logins',
'weekly_plan',
'log_total_cancel',
'cross_sell_success',
'every_cancelled',
'less_than_30_flag',
'weekly_plan_present',
'order_time_change_ever',
'order_time_change',
'total_lockers',
'lockers_present',
'log_early_deliveries',
'refrigerated_locker',
'mobile_logins',
'package_locker',
'unique_meals_purch',
'contact_service_orders',
'avg_clicks_per_visit']

# printing for the plusses 
#for val in x_variables:
    #print(f"{val} +")

#### iii) Base Model 

In [ ]:
# building a base model with a log transformed response variable


# INSTANTIATING a model type 
lm_OLS = smf.ols(formula = """log_revenue ~
                                    dum_master_classes_attended_1 +
dum_master_classes_attended_2 +
dum_master_classes_attended_3 +
dum_median_rating_2 +
dum_median_rating_3 +
dum_median_rating_4 +
dum_median_rating_5 +
dum_mail_type_unknown +
dum_mail_type_spam +
dum_mail_type_professional +
log_total_meals_ordered +
log_total_meals_ordered_without_cancel +
log_avg_prep_vid_time +
log_order_variety +
log_median_meal_rating +
orders_per_login +
meals_per_login +
avg_num_meals_per_click +
order_variety +
log_avg_num_meals_customer +
total_photos_viewed +
log_avg_time_per_site_visit +
log_contacts_w_customer_service +
mobile_number +
product_categories_viewed +
log_cancellations_before_noon +
tastes_and_preferences +
pc_logins +
weekly_plan +
log_total_cancel +
cross_sell_success +
every_cancelled +
less_than_30_flag +
weekly_plan_present +
order_time_change_ever +
order_time_change +
total_lockers +
lockers_present +
log_early_deliveries +
refrigerated_locker +
mobile_logins +
package_locker +
unique_meals_purch +
contact_service_orders +
avg_clicks_per_visit
""",data = df)

# telling Python to FIT the data to the blueprint
results = lm_OLS.fit()


# printing a summary of the results
print(results.summary())

#### iv) Final Variable Selection & Train/Test Splits 

In [ ]:
# creating new set of variables for OLS adjusted by p-value 
x_variables_OLS = ['dum_master_classes_attended_1',
'dum_master_classes_attended_2',
'dum_master_classes_attended_3',
'dum_median_rating_2',
'dum_median_rating_3',
'dum_median_rating_4',
'dum_median_rating_5',
'dum_mail_type_unknown',
'dum_mail_type_spam',
'dum_mail_type_professional',
'log_total_meals_ordered',
'log_avg_prep_vid_time',
'log_order_variety',
'log_median_meal_rating',
'log_avg_num_meals_customer',
'total_photos_viewed',
'log_contacts_w_customer_service',
'cross_sell_success',
'weekly_plan_present',
'unique_meals_purch']

#for val in x_variables_OLS:
    #print(f"{val} +")

In [ ]:
# Simple splits 

# Preparing a DataFrame based the the analysis above
ols_data = df.loc[ : , x_variables_OLS] #for OLS p-value adjusted 
data = df.loc[ : , x_variables] #non OLS p-value adjusted 


# Preparing the target variable
chef_target = df.loc[ : , 'log_revenue']

# OLS p-value x-dataset (normal Y)
x_train_OLS, x_test_OLS, y_train_OLS, y_test_OLS = train_test_split(
            ols_data,         # x-variables
            chef_target,   # y-variable
            test_size = 0.25,
            random_state = 219)

# Data without p value optimized 
x_train, x_test, y_train, y_test = train_test_split(
            data,         # x-variables
            chef_target,   # y-variable
            test_size = 0.25,
            random_state = 219)

# Data for KNN 

X_train, X_test, y_train, y_test = train_test_split(
            data,
            chef_target,
            test_size = 0.25,
            random_state = 219)


In [ ]:
# Standardized split 

# copying df 
data_scale = data.copy()

# INSTANTIATING a StandardScaler() object
scaler = StandardScaler()


# FITTING the scaler with housing_data
scaler.fit(data_scale)


# TRANSFORMING our data after fit
X_scaled = scaler.transform(data_scale)


# converting scaled data into a DataFrame
X_scaled_df = pd.DataFrame(X_scaled)


# checking the results
X_scaled_df.describe().round(2)

In [ ]:
# Standardized train-test-split
X_train_STAND, X_test_STAND, y_train_STAND, y_test_STAND = train_test_split(
            X_scaled_df,
            chef_target,
            test_size = 0.25,
            random_state = 219)

### b) OLS Model

In [ ]:
# INSTANTIATING a model object
lr = LinearRegression()


# FITTING to the training data
lr_fit = lr.fit(x_train_OLS, y_train_OLS)


# PREDICTING on new data
lr_pred = lr_fit.predict(x_test_OLS)


# SCORING the results
print('OLS Training Score :', lr.score(x_train_OLS, y_train_OLS).round(4))
print('OLS Testing Score  :', lr.score(x_test_OLS, y_test_OLS).round(4))


# saving scoring data for future use
lr_train_score = lr.score(x_train_OLS, y_train_OLS).round(4) # using R-square
lr_test_score  = lr.score(x_test_OLS, y_test_OLS).round(4)   # using R-square


# displaying and saving the gap between training and testing
print('OLS Train-Test Gap :', abs(lr_train_score - lr_test_score).round(4))
lr_test_gap = abs(lr_train_score - lr_test_score).round(4)

In [ ]:
# zipping each feature name to its coefficient
lr_model_values = zip(df[x_variables].columns,
                      lr_fit.coef_.round(decimals = 2))


# setting up a placeholder list to store model features
lr_model_lst = [('intercept', lr_fit.intercept_.round(decimals = 2))]


# printing out each feature-coefficient pair one by one
for val in lr_model_values:
    lr_model_lst.append(val)
    

# checking the results
for pair in lr_model_lst:
    print(pair)

### c) Lasso Regression Model

In [ ]:
# INSTANTIATING a model object
lasso_model = sklearn.linear_model.Lasso(alpha = 0.001,
                                         normalize = False) # default magitude


# FITTING to the training data
lasso_fit = lasso_model.fit(x_train, y_train)


# PREDICTING on new data
lasso_pred = lasso_fit.predict(x_test)


# SCORING the results
print('Lasso Training Score :', lasso_model.score(x_train, y_train).round(4))
print('Lasso Testing Score  :', lasso_model.score(x_test, y_test).round(4))


# saving scoring data for future use
lasso_train_score = lasso_model.score(x_train, y_train).round(4) 
lasso_test_score  = lasso_model.score(x_test, y_test).round(4) 


# displaying and saving the gap between training and testing
print('Lasso Train-Test Gap :', abs(lasso_train_score - lasso_test_score).round(4))
lasso_test_gap = abs(lasso_train_score - lasso_test_score).round(4)

In [ ]:
# zipping each feature name to its coefficient
lasso_model_values = zip(df[x_variables].columns, lasso_fit.coef_.round(decimals = 2))


# setting up a placeholder list to store model features
lasso_model_lst = [('intercept', lasso_fit.intercept_.round(decimals = 2))]


# printing out each feature-coefficient pair one by one
for val in lasso_model_values:
    lasso_model_lst.append(val)
    

# checking the results
for pair in lasso_model_lst:
    print(pair)

### d) ARD Model

In [ ]:
# INSTANTIATING a model object
ard_model = sklearn.linear_model.ARDRegression(normalize  = False)


# FITTING the training data
ard_fit = ard_model.fit(x_train, y_train)


# PREDICTING on new data
ard_pred = ard_fit.predict(x_test)


print('Training Score:', ard_model.score(x_train, y_train).round(4))
print('Testing Score :',  ard_model.score(x_test, y_test).round(4))


# saving scoring data for future use
ard_train_score = ard_model.score(x_train, y_train).round(4)
ard_test_score  = ard_model.score(x_test, y_test).round(4)


# displaying and saving the gap between training and testing
print('ARD Train-Test Gap :', abs(ard_train_score - ard_test_score).round(4))
ard_test_gap = abs(ard_train_score - ard_test_score).round(4)

In [ ]:
# zipping each feature name to its coefficient
ard_model_values = zip(df[x_variables].columns, ard_fit.coef_.round(decimals = 5))


# setting up a placeholder list to store model features
ard_model_lst = [('intercept', ard_fit.intercept_.round(decimals = 2))]


# printing out each feature-coefficient pair one by one
for val in ard_model_values:
    ard_model_lst.append(val)
    

# checking the results
for pair in ard_model_lst:
    print(pair)

### e) KNN Model

In [ ]:
# creating lists for training set accuracy and test set accuracy
training_accuracy = []
test_accuracy     = []


# building a visualization of 1 to 50 neighbors
neighbors_settings = range(1, 21)


for n_neighbors in neighbors_settings:
    # Building the model
    clf = KNeighborsRegressor(n_neighbors = n_neighbors)
    clf.fit(X_train, y_train)
    
    # Recording the training set accuracy
    training_accuracy.append(clf.score(X_train, y_train))
    
    # Recording the generalization accuracy
    test_accuracy.append(clf.score(X_test, y_test))


# plotting the visualization
fig, ax = plt.subplots(figsize=(12,8))
plt.plot(neighbors_settings, training_accuracy, label = "training accuracy")
plt.plot(neighbors_settings, test_accuracy, label = "test accuracy")
plt.ylabel("Accuracy")
plt.xlabel("n_neighbors")
plt.legend()
plt.show()

In [ ]:
# finding the optimal number of neighbors
opt_neighbors = test_accuracy.index(max(test_accuracy)) + 1
print(f"""The optimal number of neighbors is {opt_neighbors}""")

In [ ]:
# INSTANTIATING a KNN model object
knn_reg = KNeighborsRegressor(algorithm = 'auto',
                              n_neighbors = opt_neighbors)


# FITTING to the training data
knn_fit = knn_reg.fit(X_train, y_train)


# PREDICTING on new data
knn_reg_pred = knn_fit.predict(X_test)


# SCORING the results
print('KNN Training Score:', knn_reg.score(X_train, y_train).round(4))
print('KNN Testing Score :',  knn_reg.score(X_test, y_test).round(4))


# saving scoring data for future use
knn_reg_score_train = knn_reg.score(X_train, y_train).round(4)
knn_reg_score_test  = knn_reg.score(X_test, y_test).round(4)


# displaying and saving the gap between training and testing
print('KNN Train-Test Gap:', abs(knn_reg_score_train - knn_reg_score_test).round(4))
knn_reg_test_gap = abs(knn_reg_score_train - knn_reg_score_test).round(4)

### f) KNN Scaled Model 

In [ ]:
# creating lists for training set accuracy and test set accuracy
training_accuracy = []
test_accuracy = []


# building a visualization of 1 to 50 neighbors
neighbors_settings = range(1, 21)


for n_neighbors in neighbors_settings:
    # Building the model
    clf = KNeighborsRegressor(n_neighbors = n_neighbors)
    clf.fit(X_train_STAND, y_train_STAND)
    
    # Recording the training set accuracy
    training_accuracy.append(clf.score(X_train_STAND, y_train_STAND))
    
    # Recording the generalization accuracy
    test_accuracy.append(clf.score(X_test_STAND, y_test_STAND))


# plotting the visualization
fig, ax = plt.subplots(figsize=(12,8))
plt.plot(neighbors_settings, training_accuracy, label = "training accuracy")
plt.plot(neighbors_settings, test_accuracy,     label = "test accuracy")
plt.ylabel("Accuracy")
plt.xlabel("n_neighbors")
plt.legend()
plt.show()


# finding the optimal number of neighbors
opt_neighbors = test_accuracy.index(max(test_accuracy)) + 1
print(f"""The optimal number of neighbors is {opt_neighbors}""")

In [ ]:
# INSTANTIATING a model with the optimal number of neighbors
knn_stand = KNeighborsRegressor(algorithm = 'auto',
                                n_neighbors = opt_neighbors)


# FITTING the model based on the training data
knn_stand_fit = knn_stand.fit(X_train_STAND, y_train_STAND)


# PREDITCING on new data
knn_stand_pred = knn_stand_fit.predict(X_test_STAND)


# SCORING the results
print('KNN Training Score:', knn_stand.score(X_train_STAND, y_train_STAND).round(4))
print('KNN Testing Score :',  knn_stand.score(X_test_STAND, y_test_STAND).round(4))


# saving scoring data for future use
knn_stand_score_train = knn_stand.score(X_train_STAND, y_train_STAND).round(4)
knn_stand_score_test  = knn_stand.score(X_test_STAND, y_test_STAND).round(4)


# displaying and saving the gap between training and testing
print('KNN Train-Test Gap:', abs(knn_stand_score_train - knn_stand_score_test).round(4))
knn_stand_test_gap = abs(knn_stand_score_train - knn_stand_score_test).round(4)

## 3) Model Results

In [ ]:
# comparing results

print(f"""
Model      Train Score      Test Score      Gap
-----      -----------      ----------      ----------
OLS        {lr_train_score}           {lr_test_score}          {lr_test_gap}  
Lasso      {lasso_train_score}           {lasso_test_score}          {lasso_test_gap}
ARD        {ard_train_score}           {ard_test_score}          {ard_test_gap}
KNN        {knn_reg_score_train}           {knn_reg_score_test}          {knn_reg_test_gap}
KNN_stand {knn_stand_score_train}           {knn_stand_score_test}            {knn_stand_test_gap}
""")

NA = "NA" # defining NA for later

# creating a dictionary for model results
model_performance = {
    
    'Model Type'    : ['OLS', 'Lasso', 'ARD','KNN','KNN_stand'],
           
    'Training' : [lr_train_score, lasso_train_score,
                                   ard_train_score,knn_reg_score_train,knn_stand_score_train],
           
    'Testing'  : [lr_test_score, lasso_test_score,
                                   ard_test_score,knn_reg_score_test,knn_stand_score_test],
                    
    'Train-Test Gap' : [lr_test_gap, lasso_test_gap,
                                        ard_test_gap,knn_reg_test_gap,knn_stand_test_gap],
                    
    'Model Size' : [len(lr_model_lst), len(lasso_model_lst),
                                    len(ard_model_lst),NA,NA],
                    
    'Model' : [lr_model_lst, lasso_model_lst, ard_model_lst,NA,NA]}


# converting model_performance into a DataFrame
model_performance = pd.DataFrame(model_performance)

# showing model results
model_performance

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

<font color='red' size="100"> Final Model: ARD </font>

<br>
<font color='blue' size="70"> The ARD Coefficients can be viewed below in detail </font>

<br><hr style="height:.9px;border:none;color:#333;background-color:#333;" />

In [ ]:
# Display of ARD model coefficients
ard_model_lst